In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
def import_and_authenticate():
  !pip install -U -q PyDrive
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials

  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  return GoogleDrive(gauth)

def get_dataset(link, drive):
  fluff, id = link.split('=')
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('temp.csv')  
  dataset = pd.read_csv('temp.csv')
  return dataset

def activate_drive():
  from google.colab import drive
  drive.mount('/content/gdrive')

import matplotlib.pyplot as plt
def plot_process(history, name):
  plt.figure(figsize=(15,10))
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.plot(history.history['mean_squared_error'])
  plt.plot(history.history['val_mean_squared_error'])
  plt.title('model Description')
  plt.ylabel('Error')
  plt.xlabel('epoch')
  plt.legend(['train_loss', 'test_loss','train_mean_squared_error','test_mean_squared_error'], loc='upper left')
  plt.show()
  plt.savefig(name)

In [0]:
drive = import_and_authenticate()
#train = get_dataset('https://drive.google.com/open?id=1r895ehmjNucPkcfasZgVNKFJGjKA2AKd', drive) // Not modified
train = get_dataset('https://drive.google.com/open?id=1Y2nkm8s3FAa4feA4m_ofjeiGf6VoyQ2f',drive) # Zero-filled

In [4]:
train.head()

,date,id,cost,sold
0,2018-12-01,0,350.0,0.0
1,2018-12-01,3371,637.5,0.0
2,2018-12-01,3370,720.0,0.0
3,2018-12-01,3369,497.5,0.0
4,2018-12-01,3368,292.0,0.0


## Renaming columns

In [0]:
train.rename(columns={'date':'created_date','sold':'item_quantity'}, inplace=True)

## Date preprocessing

In [0]:
# Casting to date column created_date
train['created_date'] = pd.to_datetime(train['created_date'], format='%Y-%m-%d')

# Fetching a year
train['day_in_year'] = train['created_date'].dt.dayofyear
minim = min(train['created_date'].dt.year)
train['year'] = train['created_date'].dt.year - minim
train.head(4)

# Linerizing a day according year
lowest_day = train[train['year']==0]['day_in_year'].min()
highest_day = train[train['year']==0]['day_in_year'].max()
train['day_number'] = train['year'] * highest_day + train['day_in_year'] - lowest_day + 5

# Week Binning
train['week'] = np.int32(train['day_number'] / 7)
train['day_of_week'] = np.int32(train['day_number'] % 7)
train.drop(columns = ['created_date','day_in_year','year','day_of_week'], inplace = True)

train = train[['id','cost','day_number','week','item_quantity']]

In [7]:
train.head()

,id,cost,day_number,week,item_quantity
0,0,350.0,5,0,0.0
1,3371,637.5,5,0,0.0
2,3370,720.0,5,0,0.0
3,3369,497.5,5,0,0.0
4,3368,292.0,5,0,0.0


## Sorting our values

In [0]:
# Sorting a values
train = train.sort_values(by = ['id','day_number']).reset_index(drop = True)

In [9]:
train.head()

,id,cost,day_number,week,item_quantity
0,0,350.0,5,0,0.0
1,0,350.0,6,0,0.0
2,0,350.0,7,1,0.0
3,0,350.0,8,1,0.0
4,0,350.0,9,1,0.0


## Train & Test

In [0]:
def split_into_train_and_test(dataset):
  last_week = dataset['week'].max() # 19 week
  dataset = dataset[dataset['week'] != last_week]

  last_week = dataset['week'].max() # 18 week
  test = dataset[dataset['week'] == last_week].copy() # 18 week
  train = dataset[dataset['week'] != last_week] # All before 18 week
  return train,test

train_df, test_df = split_into_train_and_test(train)

In [11]:
train_df.tail(4)

,id,cost,day_number,week,item_quantity
636603,5036,36.0,122,17,125.0
636604,5036,36.0,123,17,40.0
636605,5036,36.0,124,17,65.0
636606,5036,36.0,125,17,59.0


In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 597135 entries, 0 to 636606
Data columns (total 5 columns):
id               597135 non-null int64
cost             597135 non-null float64
day_number       597135 non-null int64
week             597135 non-null int32
item_quantity    597135 non-null float64
dtypes: float64(2), int32(1), int64(2)
memory usage: 25.1 MB


In [13]:
test_df.head(4)

,id,cost,day_number,week,item_quantity
121,0,350.0,126,18,0.0
122,0,350.0,127,18,0.0
123,0,350.0,128,18,0.0
124,0,350.0,129,18,0.0


## Splitting into X & y

In [0]:
def split(dataset):
  return dataset[dataset.columns.values[:-1]], dataset[dataset.columns.values[-1]]
X_train, y_train = split(train_df)
X_test, y_test = split(test_df)

## Scaler class

In [0]:
from sklearn.preprocessing import MinMaxScaler
class Scaler_3d():

  def __init__(self):
    self.__X_scaler = MinMaxScaler(feature_range=(-1, 1), copy=True)
    self.__y_scaler = MinMaxScaler(feature_range=(-1, 1), copy=True)
  
  # Fitting
  def fit(self, X_train, y_train):
    x = X_train.shape[0]
    y = X_train.shape[1]
    z = X_train.shape[2]    
    
    self.__X_scaler.fit(np.reshape(X_train, (x*y,z)))
    self.__y_scaler.fit(y_train)
  
  # scaling
  def transform(self, X, y):
    
    x_dim = X.shape[0]
    y_dim = X.shape[1]
    z_dim = X.shape[2]    
    
    X_reshaped =  np.reshape(X, (x_dim * y_dim, z_dim))
    X_reshaped_scaled = self.__X_scaler.transform(X_reshaped)
    X_scaled = np.reshape(X_reshaped_scaled, (x_dim, y_dim, z_dim))
    
    y_scaled = self.__y_scaler.transform(y)
    return X_scaled, y_scaled 
  
  def inverse_transform_X(self, X):
    x_dim = X.shape[0]
    y_dim = X.shape[1]
    z_dim = X.shape[2]    
    
    X_reshaped =  np.reshape(X, (x_dim * y_dim, z_dim))
    X_reshaped_scaled = self.__X_scaler.inverse_transform(X_reshaped)
    X_scaled = np.reshape(X_reshaped_scaled, (x_dim, y_dim, z_dim))
  
  def inverse_transform_y(self, y):
    return self.__y_scaler.inverse_transform(y)
    

## Building a sliding window for train_matrix
      + One sliding window = 4 weeks = 28 days
      + frequency of discretization: 1 day
      (# Records, # Timestamps in Rec, # of vars in one timestamp).
   
     - We have a different item_id

In [0]:
rec_size = 28 # Represents amount of previous records in current record
window_size = rec_size * train.shape[1] # Amount of records in one preprocessed record
last_id = train['id'].max()

In [0]:
def train_supervised_2_time_series(dataset, last_label, amount_of_timestamps_in_a_record):
  
  X_train_ = []
  y_train_ = []
  last_label += 1 # Used to include highest label
  
  for item_id in range(0, last_label):
    # Fetching table belonging to partilular id
    temp_df = dataset[dataset['id']==item_id].copy()

    # Turn to next id if no records
    if temp_df.empty: continue

    # to numpy
    matrix = temp_df.values
    
    # Clean memory
    del temp_df
    
    for i in range(amount_of_timestamps_in_a_record, matrix.shape[0]):

      # we will append the first 28 values to X
      X_train_.append(matrix[i-amount_of_timestamps_in_a_record:i, :])
      # and 28 value will be appended to y
      y_train_.append(matrix[i, -1])
  
  return np.array(X_train_), np.reshape(np.array(y_train_),(-1,1))

In [18]:
import time
start = time.time()

X_train, y_train = train_supervised_2_time_series(train, train['id'].max(), rec_size)

end = time.time()
print(end - start)

8.706854343414307


In [19]:
X_train[0]

array([[  0., 350.,   5.,   0.,   0.],
       [  0., 350.,   6.,   0.,   0.],
       [  0., 350.,   7.,   1.,   0.],
       [  0., 350.,   8.,   1.,   0.],
       [  0., 350.,   9.,   1.,   0.],
       [  0., 350.,  10.,   1.,   0.],
       [  0., 350.,  11.,   1.,   0.],
       [  0., 350.,  12.,   1.,   0.],
       [  0., 350.,  13.,   1.,   0.],
       [  0., 350.,  14.,   2.,   0.],
       [  0., 350.,  15.,   2.,   0.],
       [  0., 350.,  16.,   2.,   0.],
       [  0., 350.,  17.,   2.,   0.],
       [  0., 350.,  18.,   2.,   0.],
       [  0., 350.,  19.,   2.,   0.],
       [  0., 350.,  20.,   2.,   0.],
       [  0., 350.,  21.,   3.,   0.],
       [  0., 350.,  22.,   3.,   0.],
       [  0., 350.,  23.,   3.,   0.],
       [  0., 350.,  24.,   3.,   0.],
       [  0., 350.,  25.,   3.,   0.],
       [  0., 350.,  26.,   3.,   0.],
       [  0., 350.,  27.,   3.,   0.],
       [  0., 350.,  28.,   4.,   0.],
       [  0., 350.,  29.,   4.,   0.],
       [  0., 350.,  30.,

In [20]:
y_train.shape

(498435, 1)

## Sliding window for test_matrix

In [0]:
test_df = pd.concat((train_df[train_df['week'] > 13],test_df)).sort_values(by = ['id','day_number']).reset_index(drop = True)

In [0]:
rec_size = 28 # Represents amount of previous records in current record
window_size = rec_size * test_df.shape[1] # Amount of records in one preprocessed record
last_id = train['id'].max()

In [23]:
import time
start = time.time()

X_test, y_test = train_supervised_2_time_series(test_df, test_df['id'].max(), rec_size)

end = time.time()
print(end - start)

5.718465566635132


## Scaling train

In [24]:
X_test.shape

(34545, 28, 5)

In [25]:
y_test.shape

(34545, 1)

In [0]:
scaler = Scaler_3d()

# train
scaler.fit(X_train, y_train)
X_train_scaled, y_train_scaled = scaler.transform(X_train, y_train)

# test
X_test_scaled, y_test_scaled = scaler.transform(X_test, y_test)

In [27]:
X_train_scaled.shape

(498435, 28, 5)

## Model building

In [28]:
model = keras.models.load_model('FORTE_LSTM_on_zero_filled_v1.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [28]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(80, input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2]), activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, implementation=1, return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False))
model.add(keras.layers.BatchNormalization(axis = -1))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(30,use_bias = True))
model.add(keras.layers.BatchNormalization(axis = -1))
model.add(keras.layers.Activation('tanh'))
model.add(keras.layers.Dense(10,use_bias = True))
model.add(keras.layers.BatchNormalization(axis = -1))
model.add(keras.layers.LeakyReLU(alpha=0.2))
model.add(keras.layers.Dense(1,use_bias = True))
model.add(keras.layers.BatchNormalization(axis = -1))
model.add(keras.layers.LeakyReLU(alpha=0.2))
model.compile(optimizer = keras.optimizers.Adam(), loss='mean_squared_error', metrics = ['mean_absolute_error','mean_squared_error'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 80)                27520     
_________________________________________________________________
batch_normalization_v1 (Batc (None, 80)                320       
_________________________________________________________________
flatten (Flatten)            (None, 80)                0         
_________________________________________________________________
dense (Dense)                (None, 30)                2430      
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 30)                120       
_________________________________________________________________
activation (Activation)      (None, 30)                0         
______________________

In [29]:
BATCH_SIZE = 32
history = model.fit(x = X_train_scaled, y = y_train_scaled, batch_size = BATCH_SIZE, epochs = 10, validation_data = (X_test_scaled, y_test_scaled))

Train on 458955 samples, validate on 34545 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
458955/458955 [==============================] - 734s 2ms/sample - loss: 0.2070 - mean_absolute_error: 0.2964 - mean_squared_error: 0.2070 - val_loss: 3.2549e-05 - val_mean_absolute_error: 0.0011 - val_mean_squared_error: 3.2549e-05
Epoch 2/10
458955/458955 [==============================] - 741s 2ms/sample - loss: 3.0516e-05 - mean_absolute_error: 0.0011 - mean_squared_error: 3.0516e-05 - val_loss: 3.1006e-05 - val_mean_absolute_error: 9.6503e-04 - val_mean_squared_error: 3.1006e-05
Epoch 3/10
458955/458955 [==============================] - 744s 2ms/sample - loss: 3.0013e-05 - mean_absolute_error: 9.8780e-04 - mean_squared_error: 3.0013e-05 - val_loss: 2.7411e-05 - val_mean_absolute_error: 0.0010 - val_mean_squared_error: 2.7411e-05
Epoch 4/10
458955/458955 [==============================] - 745s 2ms/sample - loss: 2.9964e-05 - mean_absolute_error: 0.0010 - mean_squared_error

## Model saving

In [0]:
model.save('FORTE_LSTM_on_zero_filled_v1.h5')

## Visualization

In [30]:
plot_process(history, 'Linear.png')

NameError: ignored

## Predicting

In [0]:
y_test_predicted_scaled = model.predict(X_test_scaled)
y_test_predicted = scaler.inverse_transform_y(y_test_predicted_scaled)

In [0]:
y_train_predicted_scaled = model.predict(X_train_scaled)
y_train_predicted = scaler.inverse_transform_y(y_train_predicted_scaled)

In [33]:
y_test_predicted.shape

(34545, 1)

## Sliding window -> Standard data

In [35]:
X_train[101,]

array([[  1., 530.,   5.,   0.,   0.],
       [  1., 530.,   6.,   0.,   0.],
       [  1., 530.,   7.,   1.,   0.],
       [  1., 530.,   8.,   1.,   0.],
       [  1., 530.,   9.,   1.,   0.],
       [  1., 530.,  10.,   1.,   0.],
       [  1., 530.,  11.,   1.,   0.],
       [  1., 530.,  12.,   1.,   0.],
       [  1., 530.,  13.,   1.,   0.],
       [  1., 530.,  14.,   2.,   0.],
       [  1., 530.,  15.,   2.,   0.],
       [  1., 530.,  16.,   2.,   0.],
       [  1., 530.,  17.,   2.,   0.],
       [  1., 530.,  18.,   2.,   0.],
       [  1., 530.,  19.,   2.,   0.],
       [  1., 530.,  20.,   2.,   0.],
       [  1., 530.,  21.,   3.,   0.],
       [  1., 530.,  22.,   3.,   0.],
       [  1., 530.,  23.,   3.,   0.],
       [  1., 530.,  24.,   3.,   0.],
       [  1., 530.,  25.,   3.,   0.],
       [  1., 530.,  26.,   3.,   0.],
       [  1., 530.,  27.,   3.,   0.],
       [  1., 530.,  28.,   4.,   0.],
       [  1., 530.,  29.,   4.,   0.],
       [  1., 530.,  30.,

In [49]:
# 1. Fetching X record
# 2. Incrementing time stamp to one
# 3. Merging X & y records into var dataset

(34545, 28, 5)

In [0]:
def time_series_2_supervised(window, label):
  # 1. Fetching all last X records in a train
  X_rec = window[:,-1,:]

  # 2. Incrementing time stamp to one
  def incrementing_time_stamp(X):
    X[2] += 1
    X[3] = int(X[2] / 7)
    return X

  np.apply_along_axis(incrementing_time_stamp, 1, X_rec)
  
  # 3. Merging X & y records into var dataset
  return np.concatenate((X_rec, label), axis = 1)

matrix_predicted = time_series_2_supervised(X_train, y_train_predicted)

In [0]:
train_predicted_df = pd.DataFrame(data = matrix_predicted, columns = np.append(train_df.columns.values,'item_quantity'))

In [0]:
train_predicted_df.to_csv('LSTM_train_predicted.csv')

In [0]:
matrix_predicted = time_series_2_supervised(X_test, y_test_predicted)
test_predicted_df = pd.DataFrame(data = matrix_predicted, columns = np.append(test_df.columns.values,'item_quantity'))

In [0]:
test_predicted_df.to_csv('LSTM_test_predicted.csv')

In [68]:
test_df.shape

(172725, 5)

In [69]:
test_predicted_df.shape

(34545, 6)